In [2]:
import datasets 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import os 
import jsonlines
import json
import random

c:\Users\jojod\anaconda3\envs\DL\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
dataset = datasets.load_dataset('ucberkeley-dlab/measuring-hate-speech')   
df = dataset['train'].to_pandas()
df.describe()

,comment_id,annotator_id,platform,sentiment,respect,insult,humiliate,status,dehumanize,violence,...,hatespeech,hate_speech_score,infitms,outfitms,annotator_severity,std_err,annotator_infitms,annotator_outfitms,hypothesis,annotator_age
count,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135556.00000,135556.000000,135556.000000,135556.000000,135556.000000,...,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135556.000000,135451.000000
mean,23530.416138,5567.097812,1.281352,2.954307,2.828875,2.56331,2.278638,2.698575,1.846211,1.052045,...,0.744733,-0.567428,1.034322,1.001052,-0.018817,0.300588,1.007158,1.011841,0.014589,37.910772
std,12387.194125,3230.508937,1.023542,1.231552,1.309548,1.38983,1.370876,0.898500,1.402372,1.345706,...,0.932260,2.380003,0.496867,0.791943,0.487261,0.236380,0.269876,0.675863,0.613006,11.641276
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,...,0.000000,-8.340000,0.100000,0.070000,-1.820000,0.020000,0.390000,0.280000,-1.578693,18.000000
25%,18148.000000,2719.000000,0.000000,2.000000,2.000000,2.00000,1.000000,2.000000,1.000000,0.000000,...,0.000000,-2.330000,0.710000,0.560000,-0.380000,0.030000,0.810000,0.670000,-0.341008,29.000000
50%,20052.000000,5602.500000,1.000000,3.000000,3.000000,3.00000,3.000000,3.000000,2.000000,0.000000,...,0.000000,-0.340000,0.960000,0.830000,-0.020000,0.340000,0.970000,0.850000,0.110405,35.000000
75%,32038.250000,8363.000000,2.000000,4.000000,4.000000,4.00000,3.000000,3.000000,3.000000,2.000000,...,2.000000,1.410000,1.300000,1.220000,0.350000,0.420000,1.170000,1.130000,0.449555,45.000000
max,50070.000000,11142.000000,3.000000,4.000000,4.000000,4.00000,4.000000,4.000000,4.000000,4.000000,...,2.000000,6.300000,5.900000,9.000000,1.360000,1.900000,2.010000,9.000000,0.987511,81.000000


In [11]:
df['sentiment'].value_counts()

sentiment
4.0    61435
3.0    35243
2.0    19785
0.0     9658
1.0     9435
Name: count, dtype: int64

In [4]:
len(df['text'])

135556

In [5]:
df['hate_speech_score'].head()

0   -3.90
1   -6.52
2    0.36
3    0.26
4    1.54
Name: hate_speech_score, dtype: float64

In [6]:
df['text'][df['sentiment']== 4].head()

2    Question: These 4 broads who criticize America...
4    For starters bend over the one in pink and kic...
6    Who the fuck is this insignificant simple mind...
7           Fuck off you insufferable retarded faggot.
8    OMG! The EGO's of these young, young, inexperi...
Name: text, dtype: object

In [73]:
df['text'][2]

"Question: These 4 broads who criticize America, what country did they flee to get here? And now they want to make OUR America like THEIR former HELL HOLE. I don't think so!!!!!!!!!!  Let them explain their GRATITUDE for letting them in OUR country."

In [112]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# get the given pretrained paraphrase model and the corresponding tokenizer (https://huggingface.co/humarin/chatgpt_paraphraser_on_T5_base)
paraphrase_tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")
paraphrase_model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

def get_paraphrase_batch(
    model,
    tokenizer,
    input_samples,
    n,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=256,
    device='cuda:0'):
    '''
    Input
      model: paraphraser
      tokenizer: paraphrase tokenizer
      input_samples: a batch (list) of real samples to be paraphrased
      n: number of paraphrases to get for each input sample
      for other parameters, please refer to:
          https://huggingface.co/docs/transformers/en/main_classes/text_generation#transformers.GenerationConfig
    Output: Tuple.
      synthetic_samples: a list of paraphrased samples
    '''

    # TODO: implement paraphrasing on a batch of imput samples
    synthetic_samples = []
    for sample in input_samples:  
      label = [sample['label']]*n
      # domain = [sample['domain']]*n
      inputs_ids = tokenizer(sample['review'], return_tensors='pt', padding=True, truncation=True, max_length=max_length)
      inputs_ids = inputs_ids['input_ids'].to(device)

      outputs = model.generate(inputs_ids, temperature=temperature, repetition_penalty=repetition_penalty, num_return_sequences=n, no_repeat_ngram_size=no_repeat_ngram_size,
                              max_length=max_length, diversity_penalty=diversity_penalty, do_sample=False, num_beams=2, num_beam_groups=2)
      
      reviews = tokenizer.batch_decode(outputs, skip_special_tokens=True)
      for i in range(n):
        synthetic_samples.append({'review': reviews[i], 'label': label[i]})

    return synthetic_samples

c:\Users\jojod\anaconda3\envs\DL\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [113]:
from tqdm import tqdm
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# data_dir = 'data'
# data_train_path = os.path.join(data_dir, 'train_sa.jsonl')
BATCH_SIZE = 8
N_PARAPHRASE = 2


def get_paraphrase_dataset(model, tokenizer, batch_size, n_paraphrase, inputs):
    '''
    Input
      model: paraphrase model
      tokenizer: paraphrase tokenizer
      data_path: path to the `jsonl` file of training data
      batch_size: number of input samples to be paraphrases in one batch
      n_paraphrase: number of paraphrased sequences for each sample
    Output:
      paraphrase_dataset: a list of all paraphrase samples. Do not include the original training data.
    '''
    paraphrase_dataset = []
    inputs = np.resize(inputs, (int(len(inputs)//batch_size), batch_size))

    for input_batch in tqdm(inputs):
        paraphrase_batch = get_paraphrase_batch(model, tokenizer, input_batch, n_paraphrase)
        paraphrase_dataset.extend(paraphrase_batch)

    return paraphrase_dataset

In [9]:
#Split the sentiments in 3 classes
df_sentiment_classes = df['sentiment']
df_sentiment_classes = df_sentiment_classes.replace(1, 0)
df_sentiment_classes = df_sentiment_classes.replace(2, 1)
df_sentiment_classes = df_sentiment_classes.replace(4, 3)
df['classes'] = df_sentiment_classes

#Balance the dataset
min_size = df_sentiment_classes.value_counts().min()
indexes_0 = np.random.choice(df_sentiment_classes[df_sentiment_classes == 0].index, size=min_size)
indexes_1 = np.random.choice(df_sentiment_classes[df_sentiment_classes == 1].index, size=min_size)
indexes_3 = np.random.choice(df_sentiment_classes[df_sentiment_classes == 3].index, size=min_size)

assert len(indexes_0) == len(indexes_1) == len(indexes_3) #Check if the balancing is correct

df_balanced = df.loc[np.concatenate([indexes_0, indexes_1, indexes_3])]
df_balanced = df_balanced.reset_index(drop=True)

In [10]:
original_data = []
for i in range(len(df_balanced)):
    original_data.append({'review': df_balanced['text'][i], 'label': df_balanced['classes'][i]})

In [11]:
paraphrase_dataset = get_paraphrase_dataset(paraphrase_model, paraphrase_tokenizer, BATCH_SIZE, N_PARAPHRASE, original_data)

  0%|          | 0/7159 [00:00<?, ?it/s]

c:\Users\jojod\anaconda3\envs\DL\lib\site-packages\transformers\generation\configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
100%|██████████| 7159/7159 [27:09:35<00:00, 13.66s/it]     


In [14]:
all_data = original_data+ paraphrase_dataset

# data_dir = 'data'
# # Write all the original and paraphrased data samples into training dataset
# augmented_data_train_path = os.path.join(data_dir, 'augmented_train_sa_all.jsonl')
# with jsonlines.open(augmented_data_train_path, "w") as writer:
#     writer.write_all(all_data)

# assert len(all_data) == 3 * len(original_data)

In [13]:
# data_dir = 'data'
# ori = os.path.join(data_dir, 'original_data.jsonl')
# with jsonlines.open(ori, "w") as writer:
#     writer.write_all(original_data)

In [15]:
len(original_data)

57279

In [16]:
len(paraphrase_dataset)

114544

In [17]:
len(all_data)

171823

In [21]:
# ori = os.path.join(data_dir, 'paraphrased_data.jsonl')
# with jsonlines.open(ori, "w") as writer:
#     writer.write_all(paraphrase_dataset)

In [17]:
import re
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2066"
        u"\u2069"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                "]+", re.UNICODE)
    return re.sub(emoj,'',data)

In [19]:
import json

# Define the input and output file paths
jsonl_file_path = './data/original_data.jsonl'
txt_file_path = 'output_hate.txt'

# Open the JSONL file for reading
with open(jsonl_file_path, 'r') as jsonl_file:
    # Open the TXT file for writing
    with open(txt_file_path, 'w') as txt_file:
        # Process each line in the JSONL file
        for line in jsonl_file:
            # Parse the JSON content
            record = json.loads(line)
            # Extract the desired fields (modify this as needed)
            text = record.get('review', 'N/A')
            label = record.get('label', 'N/A')
            name = remove_emojis(text)
            # Format the data for writing (modify this as needed)
            # line_to_write = f"review: {name}, label: {age}\n"
            if label == 1.0:
                line_to_write = f"{text}\n"
            # Write the formatted data to the TXT file
                txt_file.write(line_to_write)

print("Conversion from JSONL to TXT completed successfully.")


Conversion from JSONL to TXT completed successfully.


In [18]:
#open jsonl file
jsonl_file_path = './data/original_data.jsonl'
df_old = pd.read_json(jsonl_file_path, lines=True)

In [19]:
import pandas as pd

In [20]:
jsonl_file_path = './data/paraphrased_data.jsonl'
para = pd.read_json(jsonl_file_path, lines=True)

In [21]:
df_old['label'].value_counts()

label
0    38186
1    38186
Name: count, dtype: int64

In [86]:
# hate_speech = df_old[df_old['label'] == 1]['review'].to_list()

In [87]:
# hate_df = df[df['sentiment'] == 4]

In [95]:
# new_hate = hate_df['text'].to_list()

# print(len(new_hate))

# added_data = []
# for i in range(len(new_hate)):
#     if new_hate[i] not in hate_speech and len(added_data) < 19093:
#         added_data.append({'review': new_hate[i], 'label': 1.0})
    
# print(len(added_data))

61435
19093


In [22]:
original_data = []
for i in range(len(df_old)):
    original_data.append({'review': df_old['review'][i], 'label': float(df_old['label'][i])})

In [23]:
para_d = []
for i in range(len(para)):
    para_d.append({'review': para['review'][i], 'label': float(para['label'][i])})

In [114]:
# new_paraphrasing = get_paraphrase_dataset(paraphrase_model, paraphrase_tokenizer, BATCH_SIZE, N_PARAPHRASE, added_data)

  0%|          | 0/2386 [00:00<?, ?it/s]c:\Users\jojod\anaconda3\envs\DL\lib\site-packages\transformers\generation\configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
100%|██████████| 2386/2386 [5:45:44<00:00,  8.69s/it]    


In [12]:
import os
import jsonlines

In [24]:
data_dir = 'data'
all_para = para_d + original_data
ori = os.path.join(data_dir, './augmented_data.jsonl')
with jsonlines.open(ori, "w") as writer:
    writer.write_all(all_para)

In [25]:
len(all_para)

229092